In [1]:
import torch.nn as nn
import torch
import pandas as pd
import numpy as np
from models import lstm
import importlib
import torch.optim as optim
import torch.nn.functional as f

languages = 2

In [2]:
data_path = '/Users/oosv/data/csv'
data_en = pd.read_csv(f"{data_path}/english_data.csv").values.astype(np.float32)
data_es = pd.read_csv(f"{data_path}/spanish_data.csv").values.astype(np.float32)

In [3]:
print(np.shape(data_en))
print(np.shape(data_es))

(4939049, 39)
(1760399, 39)


In [4]:
importlib.reload(lstm)

<module 'models.lstm' from '/Users/oosv/prototype/models/lstm.py'>

In [5]:
model = lstm.LSTM()


In [35]:
loss_function = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.1)
#figure out a way to mix samples up and training data properly
def train(samples, labels, model, loss_function, optimizer,  epoch=2):
    for e in range(epoch):
        for sample, label in zip(samples, labels):
            model.zero_grad()
            model.hidden = model.init_hidden()
            scores = model(sample)
            loss = loss_function(scores[-1].view(1,languages), label.view(-1))
            loss.backward()
            optimizer.step()

In [6]:
data_en_sampled = np.reshape(data_en[74:], (-1,75,39))
data_es_sampled = np.reshape(data_es[74:], (-1,75,39))
en_idx = int(len(data_en_sampled)/4)
es_idx = int(len(data_es_sampled)/4)

In [42]:
samples_train = np.concatenate((data_en_sampled[en_idx:], \
                                data_es_sampled[es_idx:]), axis = 0)
samples_test_en = data_en_sampled[:en_idx]
samples_test_es = data_es_sampled[:es_idx]

labels_train = []

for l in range(len(data_en_sampled[en_idx:])):
    labels_train.append(0)

for l in range(len(data_es_sampled[es_idx:])):
    labels_train.append(1)
    
labels_test_es = []
labels_test_en = []

for l in range(len(data_en_sampled[:en_idx])):
    labels_test_en.append(0)

for l in range(len(data_es_sampled[:es_idx])):
    labels_test_es.append(1)
    
print(np.shape(samples_train))

(66994, 75, 39)


In [36]:
def shuffle_in_unison_scary(a, b):
    rng_state = np.random.get_state()
    np.random.shuffle(a)
    np.random.set_state(rng_state)
    np.random.shuffle(b)

In [89]:
shuffle_in_unison_scary(samples_train, labels_train)

samples_train = torch.from_numpy(np.array(samples_train).astype(np.float32))
labels_train = torch.tensor(labels_train, dtype=torch.float32).long()

In [38]:
samples_test_en = torch.from_numpy(np.array(samples_test_en).astype(np.float32))
labels_test_en = torch.tensor(labels_test_en, dtype=torch.float32).long()
samples_test_es = torch.from_numpy(np.array(samples_test_es).astype(np.float32))
labels_test_es = torch.tensor(labels_test_es, dtype=torch.float32).long()

In [70]:
scores = model(samples_train[0])
print(scores.size())
print(scores[-1])
print(scores[-1].view(1,2))
#print(labels[0].view(-1))

torch.Size([75, 2])
tensor([-1.1598, -0.3762], grad_fn=<SelectBackward>)
tensor([[-1.1598, -0.3762]], grad_fn=<ViewBackward>)


In [ ]:
with torch.no_grad():
    pred = [np.argmax(model(sample)[-1].view(-1).numpy()) for sample in samples_test]
    acc = []
    for pre,label in zip(pred,labels_test):
        acc.append((pre == label.numpy()))
    print(np.average(acc))
    #print(acc)

In [86]:
samples_test_en = data_en_sampled[:en_idx]
samples_test_es = data_es_sampled[:es_idx]
labels_test_es = []
labels_test_en = []

for l in range(len(samples_test_en)):
    labels_test_en.append(0)

for l in range(len(samples_test_es)):
    labels_test_es.append(1)

In [90]:
print(len(labels_test_en))
print(len(labels_test_es))

16463
5867


In [92]:
with torch.no_grad():
    pred = []
    for sample in samples_test_en:
        model.hidden = model.init_hidden()
        pred.append(np.argmax(model(sample)[-1].view(-1).numpy()))
    
    acc = []
    for pre,label in zip(pred,labels_test_en):
        acc.append((pre == label.numpy()))
    print(np.average(acc))

with torch.no_grad():
    pred = []
    for sample in samples_test_es:
        model.hidden = model.init_hidden()
        pred.append(np.argmax(model(sample)[-1].view(-1).numpy()))    
    
    acc = []
    for pre,label in zip(pred,labels_test_es):
        acc.append((pre == label.numpy()))
    print(np.average(acc))

0.7266597825426715
0.2926538264871314


In [28]:
trial = data_en_sampled[0]
print(trial)
print(np.shape(trial))
shape = np.shape(trial)

[[ 10.204676   -12.636937   -11.062954   ...  -0.01432903   0.8103626
    0.10494173]
 [ 10.2578125  -12.017317   -10.990821   ...  -0.0783997    0.57345796
    0.5007653 ]
 [ 10.246252   -11.780709   -11.856711   ...   0.18643181   0.18121667
    0.7894497 ]
 ...
 [ 10.201228   -11.100131    -6.233317   ...   0.19121367  -0.70668447
   -0.44318554]
 [ 10.150369   -12.443208    -7.726891   ...  -0.05653857  -1.0876395
   -0.28177303]
 [ 10.2936125  -12.076303    -9.61423    ...  -0.12003915  -0.86074924
   -0.12338059]]
(75, 39)


In [75]:


def _forward(vec):
    global hide
    vec = linear1(vec)
    vec = sig(vec)
    vec = linear2(vec)
    vec = linear3(vec)
    vec, hide = bilstm(vec.view([vec.size(0),1,-1]), hide)
    vec = f.log_softmax(vec)
    return vec

hidden_dim = 128
hide = (torch.zeros(2, 1,hidden_dim // 2),
                torch.zeros(2, 1, hidden_dim // 2))
linear1 = nn.Linear(shape[1], hidden_dim)
sig1 = nn.Sigmoid()
linear2 = nn.Linear(hidden_dim, hidden_dim)
sig2 = nn.Sigmoid()
linear3 = nn.Linear(hidden_dim, hidden_dim)
sig3 = nn.Sigmoid()
bilstm = nn.LSTM(hidden_dim, hidden_dim // 2, bidirectional = True)

In [76]:
t = (torch.tensor(trial))
print(t)
print(t.size())
print(_forward(t))

tensor([[ 10.2047, -12.6369, -11.0630,  ...,  -0.0143,   0.8104,   0.1049],
        [ 10.2578, -12.0173, -10.9908,  ...,  -0.0784,   0.5735,   0.5008],
        [ 10.2463, -11.7807, -11.8567,  ...,   0.1864,   0.1812,   0.7894],
        ...,
        [ 10.2012, -11.1001,  -6.2333,  ...,   0.1912,  -0.7067,  -0.4432],
        [ 10.1504, -12.4432,  -7.7269,  ...,  -0.0565,  -1.0876,  -0.2818],
        [ 10.2936, -12.0763,  -9.6142,  ...,  -0.1200,  -0.8607,  -0.1234]])
torch.Size([75, 39])
tensor([[[-4.2746, -4.3007, -4.3653,  ..., -4.3472, -4.2973, -4.3267]],

        [[-4.2543, -4.2994, -4.3460,  ..., -4.3404, -4.2912, -4.3465]],

        [[-4.2472, -4.3221, -4.3295,  ..., -4.3357, -4.2890, -4.3437]],

        ...,

        [[-4.2974, -4.3331, -4.3542,  ..., -4.3020, -4.3277, -4.3142]],

        [[-4.2992, -4.3244, -4.3568,  ..., -4.2952, -4.3450, -4.2846]],

        [[-4.3270, -4.3164, -4.3662,  ..., -4.2900, -4.4134, -4.2802]]],
       grad_fn=<LogSoftmaxBackward>)


/Users/oosv/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  # Remove the CWD from sys.path while we load stuff.


In [123]:
ten = torch.randn((10, 75, 39))*16
print(ten[0])

tensor([[ 10.9098, -26.2261,  -7.2046,  ...,  10.0380,  32.8245, -11.2466],
        [  0.9265,  -9.7480,   8.5806,  ...,   4.2106,  23.7612,  16.3050],
        [  6.3439,   3.9637,  10.1638,  ..., -35.1688,   0.5335,   4.3044],
        ...,
        [  6.0424,  18.1639,  -1.0022,  ...,  28.7140,   2.8837,   4.4009],
        [-31.0679,  23.2379,  19.6940,  ...,   4.2389, -36.0481, -41.4112],
        [ 14.5091,   1.9417, -36.9672,  ...,  -9.8842,  31.5563, -14.5995]])


In [129]:
linear1 = nn.Linear(39, 30)
out = linear1(ten)
tan = nn.Tanh()
tan_out = tan(out)
sig_out = sig1(out)
print(ten[0])
print(out[0])
print(tan_out[0])
print(sig_out[0])

tensor([[ 10.9098, -26.2261,  -7.2046,  ...,  10.0380,  32.8245, -11.2466],
        [  0.9265,  -9.7480,   8.5806,  ...,   4.2106,  23.7612,  16.3050],
        [  6.3439,   3.9637,  10.1638,  ..., -35.1688,   0.5335,   4.3044],
        ...,
        [  6.0424,  18.1639,  -1.0022,  ...,  28.7140,   2.8837,   4.4009],
        [-31.0679,  23.2379,  19.6940,  ...,   4.2389, -36.0481, -41.4112],
        [ 14.5091,   1.9417, -36.9672,  ...,  -9.8842,  31.5563, -14.5995]])
tensor([[ 16.9438,  -6.1944,  -5.3272,  ...,  -0.9229,   7.5183,  23.8152],
        [  3.1782,  -4.2258, -10.5094,  ...,  -4.9830,   0.5797,   0.8872],
        [-13.8354, -11.5207,  -7.4895,  ...,   6.7079,   2.8402,   0.5392],
        ...,
        [  7.2859,   3.0824,  12.2979,  ...,   6.9271,  -2.2018, -17.5132],
        [  9.7972,  -7.8304,  -3.7164,  ...,   2.6103,   1.0332,   1.9408],
        [-13.6335, -22.8914, -18.1936,  ..., -10.4943,  12.6758,  -6.2633]],
       grad_fn=<SelectBackward>)
tensor([[ 1.0000, -1.0000, 

In [46]:
np.average([0,0,0,0,0] == 0)

0.0

In [111]:
from models import lstm
import importlib
importlib.reload(lstm)

model = lstm.LSTM()
model.train(samples_train[0:10], labels_train[0:10])

torch.Size([75, 128])
torch.Size([75, 128])
torch.Size([75, 128])
torch.Size([75, 128])
torch.Size([75, 128])
torch.Size([75, 128])
torch.Size([75, 128])
torch.Size([75, 128])
torch.Size([75, 128])
torch.Size([75, 128])


In [112]:
with torch.no_grad():
    print(model(samples_train[0])[-1])
    print(labels_train[0])

torch.Size([75, 128])
tensor([-3.0136, -0.0504])
tensor(0)


In [4]:
vec = f.log_softmax(torch.tensor([1,1], dtype=torch.float32))

/Users/oosv/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  """Entry point for launching an IPython kernel.


In [5]:
vec

tensor([-0.6931, -0.6931])